# LAB6.03 Working with APIs  
## Instructions:  
Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company.  
Do it for all the flights between two dates.

In [1]:
import requests
import getpass
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
# establish connection to the skyscanner API
rapidAPIkey = getpass.getpass()
# 
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"
headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
           'x-rapidapi-key': rapidAPIkey}

In [3]:
# define a function to return the city codes
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"

    params = {"query":city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
           'x-rapidapi-key': rapidAPIkey}

    try:
        response = requests.get(url, headers = headers, params = params)
        city_code = response.json()['Places'][0]['PlaceId']
         # city_code = city_code.split('-')[0]
        return city_code
    
    except:
        print("Invalid input. Please input a country name e.g. 'Sydney'.")

In [4]:
# define a function to return the flights from a departure point to a destination on a given date
def flights_by_date(departure = 'Berlin', arrival = 'Vienna', date = '2021-12-31'):
    # derive city codes to modify the date search url
    depart = city_code(departure)
    arrive = city_code(arrival) 
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{depart}/{arrive}/{date}"
    
    # define parameters to request data on
    params = {f"inboundpartialdate":{date}}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
           'x-rapidapi-key': rapidAPIkey}
    try:
        # make request to API for json data 
        response = requests.get(url, headers=headers, params=params)
        # return the json_normalized data
        # remember pd.json_normalize() essentially assigns each key from the json list of dictionaries as a column header, the values as rows.
        return pd.json_normalize(response.json())
    except:
        print("Invalid input. Please try again.")

In [5]:
# define a function to return a list of dates given a start and an end date.
def flight_days (departure, arrival, start, end):
    # get all the dates from between the start and end dates
    dates = pd.Series(pd.date_range(start, end,freq='d').format())
    # use dictionary comprehension to get a dictionary of the date and the available flights for each date in the list of dates provided by get_dates()
    return {date:flights_by_date(departure, arrival, date) for date in dates}, dates


In [53]:
X , dates = flight_days(departure = 'Berlin', arrival = 'Vienna', start = '2021-10-01', end = "2021-10-03")
# quotes = []
# carriers = []
# places = []


{'2021-10-01':                                               Quotes  \
 0  [{'QuoteId': 1, 'MinPrice': 55, 'Direct': Fals...   
 
                                             Carriers  \
 0  [{'CarrierId': 50441, 'Name': 'easyJet'}, {'Ca...   
 
                                               Places  \
 0  [{'Name': 'Berlin Brandenburg', 'Type': 'Stati...   
 
                                           Currencies  
 0  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...  ,
 '2021-10-02':                                               Quotes  \
 0  [{'QuoteId': 1, 'MinPrice': 45, 'Direct': Fals...   
 
                                             Carriers  \
 0  [{'CarrierId': 1090, 'Name': 'Ryanair'}, {'Car...   
 
                                               Places  \
 0  [{'Name': 'Berlin Brandenburg', 'Type': 'Stati...   
 
                                           Currencies  
 0  [{'Code': 'USD', 'Symbol': '$', 'ThousandsSepa...  ,
 '2021-10-03':                                    

,Prices,Carrier
0,37,easyJet
1,91,Ryanair
2,55,easyJet
3,85,Ryanair
4,45,Ryanair
5,208,Austrian Airlines
6,37,easyJet
7,91,Ryanair


In [57]:
# define a function to return the prices, the departure airport, the arrival airport and the carrier in a given list of dates.
def flight_info(departure = 'Berlin', arrival = 'Vienna', start = '2021-10-01', end = "2021-10-03"):
    # get all dates between the start and end dates
    flights, dates = flight_days(departure, arrival, start, end)

    # create a loop to run pd.json_normalize() on the json data assigned to each key in flights.
    for date in dates:
        quotes = pd.json_normalize(flights[date]['Quotes'][0])
        carriers = pd.json_normalize(flights[date]['Carriers'][0])
        places = pd.json_normalize(flights[date]['Places'][0])

    for date in dates:
        for quote in X[date]['Quotes']: 
            quotes = quotes.append(pd.json_normalize(quote))

        for carrier in X[date]['Carriers']:
            carriers = carriers.append(pd.json_normalize(carrier))

        for place in X[date]['Places']:
            places = places.append(pd.json_normalize(place))

    x = pd.concat([quotes['MinPrice'],carriers['Name']], axis = 1).reset_index()
    x.drop('index',axis=1, inplace = True)
    x.rename(columns = {"MinPrice":"Prices","Name":"Carrier"}, inplace = True)
    x = pd.concat([x, places['Name']], axis = 1)


    

In [58]:
x = flight_info()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects